In [100]:
#%% Importar librerias
import pyodbc
import pandas as pd
#%% Conectar a la base de datos

In [101]:
#%%RUTA DEL ARCHIVO
ruta_rflex = r"C:\Users\bgacitua\Desktop\Reportes Corrección de Marcas\Reportes_marcas_rflex\reporte_turnos_sin_marcas_20250905075439.xlsx"
path_marcas_sin_turno = r"C:\Users\bgacitua\Desktop\Reportes Corrección de Marcas\Reportes_marcas_rflex\xsgQVdal_reporte_marca_sin_turno.xlsx"

In [102]:
#%% Configuración de conexión
SERVER = '192.9.200.84' 
DATABASE = 'MorphoManager'  
USERNAME = 'rrhh_morpho'  
PASSWORD = 'PQ90@ZT*'  

#%% Cadena de conexión para SQL Server
try:
    connection_string = (
        'DRIVER={ODBC Driver 17 for SQL Server};'
        f'SERVER={SERVER};'
        f'DATABASE={DATABASE};'
        f'UID={USERNAME};'
        f'PWD={PASSWORD};'
        'TrustServerCertificate=yes;'  # Para evitar problemas con certificados SSL
    )
    
    cnxn = pyodbc.connect(connection_string)
    print("¡Conexión a SQL Server establecida exitosamente! Lista para múltiples consultas.")

except pyodbc.Error as ex:
    sqlstate = ex.args[0]
    print(f"Error al conectar a SQL Server: {ex}")
    if sqlstate == '28000':
        print("Causa probable: Error de autenticación (usuario o contraseña incorrectos).")
    elif sqlstate == '08001':
        print("Causa probable: El servidor SQL Server no está accesible, el nombre del servidor es incorrecto, o el puerto (1433 por defecto) está bloqueado por un firewall.")
    else:
        print("Consulta el mensaje de error para más detalles.")


¡Conexión a SQL Server establecida exitosamente! Lista para múltiples consultas.


In [103]:
# import datetime
# fecha_actual_py = datetime.datetime.now()
# fecha_inicio_sql = fecha_actual_py.replace(day=1).strftime('%Y-%m-%d')  # Primer día del mes actual
# fecha_fin_sql = fecha_actual_py.strftime('%Y-%m-%d')  # Fecha actual

In [104]:
# %%
import datetime
fecha_actual_py = datetime.datetime.now()
fecha_inicio_sql = fecha_actual_py.replace(day=1).strftime('%Y-%m-%d')  # Primer día del mes actual
fecha_fin_sql = fecha_actual_py.strftime('%Y-%m-%d')  # Fecha actual

print(f"Fecha inicio: {fecha_inicio_sql}")
print(f"Fecha fin: {fecha_fin_sql}")
#%%   
# --- 3. Bloque para Consultas Iterativas y Transformación a DataFrame ---
# Puedes cambiar la consulta SQL aquí cada vez que quieras probar una nueva.
sql_query = f"""
-- Versión optimizada con rango de fechas automático
-- Las fechas se calculan automáticamente desde Python
DECLARE @fecha_inicio DATE = '{fecha_inicio_sql}';
DECLARE @fecha_fin DATE = '{fecha_fin_sql}';

WITH fechas_rango AS (
    -- Generar secuencia de fechas entre inicio y fin
    SELECT @fecha_inicio AS [FECHA_SANTIAGO]
    UNION ALL
    SELECT DATEADD(DAY, 1, [FECHA_SANTIAGO])
    FROM fechas_rango
    WHERE DATEADD(DAY, 1, [FECHA_SANTIAGO]) <= @fecha_fin
),
todos_trabajadores AS (
    -- Obtener todos los trabajadores activos
    SELECT DISTINCT 
        u.[ID] AS [USER_ID],
        u.[FIRSTNAME],
        u.[LASTNAME],
        u.[EMPLOYEEID]
    FROM [MorphoManager].[dbo].[User_] AS u
    WHERE u.[EMPLOYEEID] IS NOT NULL 
    AND u.[EMPLOYEEID] != ''  -- Filtrar empleados válidos
),
combinaciones_trabajador_fecha AS (
    -- Crear matriz completa trabajador x fecha
    SELECT 
        t.[USER_ID],
        t.[FIRSTNAME],
        t.[LASTNAME], 
        t.[EMPLOYEEID],
        f.[FECHA_SANTIAGO]
    FROM todos_trabajadores t
    CROSS JOIN fechas_rango f
)
SELECT 
    -- Información del trabajador
    ctf.[FIRSTNAME],
    ctf.[LASTNAME],
    ctf.[EMPLOYEEID],
    ctf.[FECHA_SANTIAGO],
    
    -- Conteo de registros por tipo (0 si no hay marcas)
    COALESCE(SUM(CASE WHEN m.[FUNCTIONKEYTEXT] = 'IN' THEN 1 ELSE 0 END), 0) AS [KEY_IN],
    COALESCE(SUM(CASE WHEN m.[FUNCTIONKEYTEXT] = 'OUT' THEN 1 ELSE 0 END), 0) AS [KEY_OUT],
    COALESCE(SUM(CASE WHEN m.[FUNCTIONKEYTEXT] = 'No Key' THEN 1 ELSE 0 END), 0) AS [NO_KEY],
    
    -- Estado de asistencia
    CASE 
        WHEN COUNT(m.[ID]) = 0 THEN 'Ausente'
        WHEN COUNT(m.[ID]) >= 1 THEN 'Asiste'
    END AS [ESTADO_ASISTENCIA],
    
    -- Información adicional
    COALESCE(COUNT(DISTINCT bd.[NAME_]), 0) AS [CANTIDAD_DISPOSITIVOS],
    COALESCE(COUNT(m.[ID]), 0) AS [TOTAL_REGISTROS],
    
    -- Información de horarios (opcional)
    COALESCE(MIN(CAST(m.[LOGDATETIME] AT TIME ZONE 'UTC' AT TIME ZONE 'Pacific SA Standard Time' AS TIME)), '00:00:00') AS [PRIMERA_MARCA],
    COALESCE(MAX(CAST(m.[LOGDATETIME] AT TIME ZONE 'UTC' AT TIME ZONE 'Pacific SA Standard Time' AS TIME)), '00:00:00') AS [ULTIMA_MARCA]
    
FROM combinaciones_trabajador_fecha ctf
LEFT JOIN [MorphoManager].[dbo].[AccessLog] AS m 
    ON ctf.[USER_ID] = m.[USERID]
    AND ctf.[FECHA_SANTIAGO] = CAST(m.[LOGDATETIME] AT TIME ZONE 'UTC' AT TIME ZONE 'Pacific SA Standard Time' AS DATE)
    AND CAST(m.[LOGDATETIME] AT TIME ZONE 'UTC' AT TIME ZONE 'Pacific SA Standard Time' AS DATE) BETWEEN @fecha_inicio AND @fecha_fin
LEFT JOIN [MorphoManager].[dbo].[BiometricDevice] AS bd 
    ON m.[MORPHOACCESSID] = bd.[ID]
    
GROUP BY 
    ctf.[FIRSTNAME],
    ctf.[LASTNAME],
    ctf.[EMPLOYEEID],
    ctf.[FECHA_SANTIAGO],
    ctf.[USER_ID]
    
ORDER BY 
    ctf.[LASTNAME], ctf.[FIRSTNAME], ctf.[FECHA_SANTIAGO] DESC
OPTION (MAXRECURSION 366);  -- Permite hasta 366 días en la recursión
"""

Fecha inicio: 2025-09-01
Fecha fin: 2025-09-05


In [105]:
#%% TRANSFORMAMOS CONSULTA EN DATAFRAME PARA TRABAJAR CON PANDAS

try:
    # Utiliza pandas.read_sql para ejecutar la consulta con la conexión ya abierta
    df_resultado = pd.read_sql(sql_query, cnxn)

    print("\n--- ¡Consulta ejecutada y DataFrame creado! ---")
    #print(f"Consulta ejecutada:\n{sql_query.strip()}") # strip() para limpiar espacios al inicio/fin

    print("\nPrimeras 5 filas del DataFrame:")
    print(df_resultado.head())

    print(f"\nNúmero de filas y columnas: {df_resultado.shape}")
    print("Columnas y tipos de datos:")
    df_resultado.info() # df.info() no retorna, solo imprime, por eso no se usa print()

except pyodbc.Error as ex:
    # Manejo de errores de consulta (ej. tabla no existe, sintaxis SQL incorrecta)
    print(f"Error al ejecutar la consulta SQL: {ex}")
    print("Por favor, revisa tu consulta SQL o si la tabla/columna existe y tienes permisos.")
except Exception as e:
    # Manejo de cualquier otro error inesperado
    print(f"Ocurrió un error inesperado durante el procesamiento: {e}")

# NOTA: La conexión NO se cierra aquí. Se mantiene abierta para la siguiente consulta.

#%% SEPARAR DV DEL RUT
# Paso 1: Identificar las filas que contienen el delimitador "-"
tiene_delimitador = df_resultado['EMPLOYEEID'].astype(str).str.contains('-')

# Paso 2: Crear la nueva columna con los valores después del delimitador
df_resultado['ID_ADICIONAL'] = df_resultado['EMPLOYEEID'].astype(str).str.split('-').str[1].fillna('')

# Paso 3: Actualizar la columna EMPLOYEEID para mantener solo la parte antes del delimitador
df_resultado.loc[tiene_delimitador, 'EMPLOYEEID'] = df_resultado.loc[tiene_delimitador, 'EMPLOYEEID'].astype(str).str.split('-').str[0]

try:
    df_resultado['EMPLOYEEID'] = pd.to_numeric(df_resultado['EMPLOYEEID'], errors='coerce')
    # errors='coerce' convertirá los valores que no puedan ser convertidos a números en NaN
    print("\n'EMPLOYEEID' se convirtió a tipo numérico (NaN si no pudo convertir).")
except Exception as e:
    print(f"\nAdvertencia: No se pudo convertir 'EMPLOYEEID' a numérico debido a: {e}")

# Mostrar el DataFrame después de la transformación
print("\nDataFrame después de la transformación:")
print(df_resultado[['EMPLOYEEID', 'ID_ADICIONAL']].head(5))

df_resultado.info()
#%%CREAR COLUMNA DE BUSQUEDA RUT + FECHA

df_resultado['EMPLOYEEID_STR'] = df_resultado['EMPLOYEEID'].astype(str) 
df_resultado['EMPLOYEEID_STR'] = df_resultado['EMPLOYEEID_STR'].str.replace(r'\.0$', '', regex=True)
#df_resultado['EMPLOYEEID'] = df_resultado['EMPLOYEEID'].astype(int) 
df_resultado['FECHA_SANTIAGO_STR'] = df_resultado['FECHA_SANTIAGO'].astype(str)
df_resultado.info()
df_resultado['BusquedaRUT'] = df_resultado['EMPLOYEEID_STR'] + ";" + df_resultado['FECHA_SANTIAGO_STR']

print(df_resultado)

C:\Users\bgacitua\AppData\Local\Temp\ipykernel_28104\470403224.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_resultado = pd.read_sql(sql_query, cnxn)



--- ¡Consulta ejecutada y DataFrame creado! ---

Primeras 5 filas del DataFrame:
   FIRSTNAME LASTNAME  EMPLOYEEID FECHA_SANTIAGO  KEY_IN  KEY_OUT  NO_KEY  \
0  Yessenia   Abanto   25022430-3     2025-09-05       0        0       0   
1  Yessenia   Abanto   25022430-3     2025-09-04       0        0       0   
2  Yessenia   Abanto   25022430-3     2025-09-03       0        0       0   
3  Yessenia   Abanto   25022430-3     2025-09-02       0        0       0   
4  Yessenia   Abanto   25022430-3     2025-09-01       0        0       0   

  ESTADO_ASISTENCIA  CANTIDAD_DISPOSITIVOS  TOTAL_REGISTROS PRIMERA_MARCA  \
0           Ausente                      0                0      00:00:00   
1           Ausente                      0                0      00:00:00   
2           Ausente                      0                0      00:00:00   
3           Ausente                      0                0      00:00:00   
4           Ausente                      0                0      00:00

In [106]:
#%%Info del Dataframe SQL
df_resultado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3775 entries, 0 to 3774
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   FIRSTNAME              3775 non-null   object 
 1   LASTNAME               3775 non-null   object 
 2   EMPLOYEEID             3645 non-null   float64
 3   FECHA_SANTIAGO         3775 non-null   object 
 4   KEY_IN                 3775 non-null   int64  
 5   KEY_OUT                3775 non-null   int64  
 6   NO_KEY                 3775 non-null   int64  
 7   ESTADO_ASISTENCIA      3775 non-null   object 
 8   CANTIDAD_DISPOSITIVOS  3775 non-null   int64  
 9   TOTAL_REGISTROS        3775 non-null   int64  
 10  PRIMERA_MARCA          3775 non-null   object 
 11  ULTIMA_MARCA           3775 non-null   object 
 12  ID_ADICIONAL           3775 non-null   object 
 13  EMPLOYEEID_STR         3775 non-null   object 
 14  FECHA_SANTIAGO_STR     3775 non-null   object 
 15  Busq

In [107]:
#%%IMPORTAR REPORTE RFLEX A TRAVÉS DE LA RUTA
#SOLO SE DEBE COPIAR LA RUTA Y EJECUTAR TODO EL CÓDIGO

ruta_excel_ruts = ruta_rflex
nombre_columna_ruts_excel = 'RUT'

# --- 2. Leer el listado de RUTs desde el archivo Excel ---
try:
    df_ruts_excel = pd.read_excel(ruta_excel_ruts)
    
    print(f"\nArchivo Excel leído exitosamente desde '{ruta_excel_ruts}'")
    print(f"Dimensiones del archivo: {df_ruts_excel.shape}")
    print("\nPrimeras filas del archivo original:")
    print(df_ruts_excel.head())
    
    # --- 3. Procesar la columna RUT para separar RUT y DV ---
    print(f"\nProcesando separación de RUT y DV...")
    
    # Convertir la columna RUT a string para poder trabajar con ella
    df_ruts_excel[nombre_columna_ruts_excel] = df_ruts_excel[nombre_columna_ruts_excel].astype(str)
    
    # Verificar si contiene el delimitador "-"
    tiene_delimitador = df_ruts_excel[nombre_columna_ruts_excel].str.contains('-', na=False)
    
    if tiene_delimitador.any():
        # Separar RUT y DV usando el delimitador "-"
        rut_split = df_ruts_excel[nombre_columna_ruts_excel].str.split('-', expand=True)
        
        # Actualizar la columna RUT con solo la parte numérica
        df_ruts_excel[nombre_columna_ruts_excel] = rut_split[0]
        
        # Crear la columna DV con el dígito verificador
        df_ruts_excel['DV'] = rut_split[1].fillna('')
        
        print(f"✅ Separación completada:")
        print(f"   - Registros con delimitador procesados: {tiene_delimitador.sum()}")
        print(f"   - RUT: contiene solo números")
        print(f"   - DV: nueva columna creada con dígitos verificadores")
    else:
        # Si no hay delimitador, crear columna DV vacía
        df_ruts_excel['DV'] = ''
        print("ℹ️  No se encontraron delimitadores '-' en la columna RUT")
    
    # Mostrar resultado después del procesamiento
    print("\nPrimeras filas después del procesamiento:")
    print(df_ruts_excel[['RUT', 'DV']].head(10))
    
    # Convertir RUT a entero para compatibilidad
    try:
        df_ruts_excel['RUT'] = pd.to_numeric(df_ruts_excel['RUT'], errors='coerce')
        print(f"✅ Columna RUT convertida a tipo numérico")
    except Exception as e:
        print(f"⚠️  Advertencia al convertir RUT a numérico: {e}")

except FileNotFoundError:
    print(f"❌ Error: El archivo Excel '{ruta_excel_ruts}' no se encontró.")
    print("   Asegúrate de la ruta y el nombre del archivo.")
    df_ruts_excel = pd.DataFrame()  # DataFrame vacío para evitar errores
except KeyError:
    print(f"❌ Error: La columna '{nombre_columna_ruts_excel}' no se encontró en el archivo Excel.")
    print("   Revisa el encabezado del archivo.")
    df_ruts_excel = pd.DataFrame()
except Exception as e:
    print(f"❌ Error inesperado al leer el archivo Excel: {e}")
    df_ruts_excel = pd.DataFrame()


Archivo Excel leído exitosamente desde 'C:\Users\bgacitua\Desktop\Reportes Corrección de Marcas\Reportes_marcas_rflex\reporte_turnos_sin_marcas_20250905075439.xlsx'
Dimensiones del archivo: (46, 10)

Primeras filas del archivo original:
          RUT                     Nombre          Unidad  \
0  16418842-6     Paulina González Muñoz        Customer   
1  25322441-K  Dariana Devies Castellano  Coordinador(a)   
2  12857871-4       Sixto Beiza Mardones        Operario   
3  27064835-5      Germán Leyes Portilla        Operario   
4  18906244-3           Marco Rojas Mora        Operario   

                              Puesto       Fecha Tipo de turno  \
0  Administración Gral De Producción  2025-09-04             T   
1                  Comercial Sabores  2025-09-04             T   
2            Esencias y Env. Sabores  2025-09-04             T   
3                  Planta Fragancias  2025-09-04             T   
4                  Planta Fragancias  2025-09-04             T   

  En

In [108]:
#%% Procesar columnas y crear clave de búsqueda
# Convertir RUT a string para crear la clave de búsqueda
df_ruts_excel['RUT'] = df_ruts_excel['RUT'].astype(str)

print("Información del DataFrame después del procesamiento:")
df_ruts_excel.info()

# Crear columna de búsqueda RUT + Fecha
df_ruts_excel['Busqueda'] = df_ruts_excel['RUT'] + ";" + df_ruts_excel["Fecha"]

print("\nDataFrame con columna de búsqueda creada:")
print(df_ruts_excel[['RUT', 'DV', 'Fecha', 'Busqueda']].head())
print(f"\nTotal de registros: {len(df_ruts_excel)}")

Información del DataFrame después del procesamiento:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   RUT                 46 non-null     object
 1   Nombre              46 non-null     object
 2   Unidad              46 non-null     object
 3   Puesto              46 non-null     object
 4   Fecha               46 non-null     object
 5   Tipo de turno       46 non-null     object
 6   Entrada programada  46 non-null     object
 7   Salida programada   46 non-null     object
 8   Marca de entrada    46 non-null     object
 9   Marca de salida     46 non-null     object
 10  DV                  46 non-null     object
dtypes: object(11)
memory usage: 4.1+ KB

DataFrame con columna de búsqueda creada:
        RUT DV       Fecha             Busqueda
0  16418842  6  2025-09-04  16418842;2025-09-04
1  25322441  K  2025-09-04  25322441;202

In [109]:
#%%IMPORTAR NUMPY
import numpy as np

#%%

df_ruts_excel['Marca de entrada'] = df_ruts_excel['Marca de entrada'].replace("--:--", np.nan)
df_ruts_excel['Marca de salida'] = df_ruts_excel['Marca de salida'].replace("--:--", np.nan)

df_ruts_excel.isna()
print(df_ruts_excel)

#%%Traer Key de Marca

# Columnas a incorporar al nuevo DF.
columnas_a_traer = ['KEY_IN', 'KEY_OUT', 'NO_KEY', 'PRIMERA_MARCA', 'ULTIMA_MARCA']

# Realizar el merge
# left_on: la columna de df_ruts_excel (izquierda) para la coincidencia
# right_on: la columna de df_resultado (derecha) para la coincidencia
# how='left': asegura que todas las filas de df_ruts_excel se mantengan.
# Si no hay coincidencia en df_resultado, las nuevas columnas tendrán NaN.

df_ruts_excel_con_datos = pd.merge(
    df_ruts_excel,           # DataFrame de la izquierda (el que quieres enriquecer)
    df_resultado[columnas_a_traer + ['BusquedaRUT']], # DataFrame de la derecha (solo las columnas que necesitas + la de búsqueda)
    left_on='Busqueda',      # Columna para buscar en df_ruts_excel
    right_on='BusquedaRUT',  # Columna para buscar en df_resultado
    how='left'               # Tipo de merge: 'left' para mantener todas las filas de df_ruts_excel
)

df_ruts_excel_con_datos = df_ruts_excel_con_datos.drop(columns=['BusquedaRUT'])
print(df_ruts_excel_con_datos)

         RUT                           Nombre              Unidad  \
0   16418842           Paulina González Muñoz            Customer   
1   25322441        Dariana Devies Castellano      Coordinador(a)   
2   12857871             Sixto Beiza Mardones            Operario   
3   27064835            Germán Leyes Portilla            Operario   
4   18906244                 Marco Rojas Mora            Operario   
5   20053662            Jazmín Castillo Ponce           Asistente   
6   17841404          Fernando Magaña Cancino        Especialista   
7   20761765            Alejandro Ayala Salas            Operario   
8   13546003            Jose San Martin Silva            Jefatura   
9    9093920          Ximena Guerrero Quezada            Jefatura   
10  11524484            Manuel Gamboa Ramirez            Jefatura   
11  17541088            Alejandro Silva Lavin          Supervisor   
12  16357908           Marco Rioseco Gallardo        Especialista   
13   9583064          Nelson Sepul

In [110]:
#%%CRUCE DE DATOS

df_rellenar = df_ruts_excel_con_datos.copy()

#Diccionario para marcas de entrada
mapa_entrada = df_resultado.set_index('BusquedaRUT')['PRIMERA_MARCA'].to_dict()

#Diccionario para marcas de salida
mapa_salida = df_resultado.set_index('BusquedaRUT')['ULTIMA_MARCA'].to_dict()

print("\nMapa de Entrada:\n", mapa_entrada) 
print("\nMapa de Salida:\n", mapa_salida)  

#Buscar Marcas de Entrada
valores_entrada_buscados = df_rellenar['Busqueda'].map(mapa_entrada)

#Reemplazar valores de entrada
df_rellenar['Marca de entrada Generados'] = df_rellenar['Marca de entrada'].fillna(valores_entrada_buscados)

# Procesar Marca de salida
valores_salida_buscados = df_rellenar['Busqueda'].map(mapa_salida)
df_rellenar['Marca de salida Generados'] = df_rellenar['Marca de salida'].fillna(valores_salida_buscados)

print("\nDataFrame Principal (después del BUSCARV condicional):\n", df_rellenar)

print("\nConteo de NaNs después del proceso:")
print(df_rellenar.isna().sum())

#df_rellenar["Marca de entrada"] = df_rellenar["Marca de entrada"].str.strip()

#df_rellenar["Marca de entrada"] = df_rellenar["Marca de entrada"].replace("00:00:00", "Sin marca")

#print(df_rellenar)


Mapa de Entrada:
 {'25022430;2025-09-05': datetime.time(0, 0), '25022430;2025-09-04': datetime.time(0, 0), '25022430;2025-09-03': datetime.time(0, 0), '25022430;2025-09-02': datetime.time(0, 0), '25022430;2025-09-01': datetime.time(0, 0), '12357791;2025-09-05': datetime.time(1, 2, 20), '12357791;2025-09-04': datetime.time(5, 49, 54), '12357791;2025-09-03': datetime.time(0, 56, 4), '12357791;2025-09-02': datetime.time(1, 3, 59), '12357791;2025-09-01': datetime.time(19, 38, 55), '14194403;2025-09-05': datetime.time(7, 54, 24), '14194403;2025-09-04': datetime.time(7, 42, 26), '14194403;2025-09-03': datetime.time(7, 43, 13), '14194403;2025-09-02': datetime.time(7, 45, 20), '14194403;2025-09-01': datetime.time(7, 36, 17), '19792744;2025-09-05': datetime.time(7, 49, 43), '19792744;2025-09-04': datetime.time(7, 51, 40), '19792744;2025-09-03': datetime.time(7, 49, 58), '19792744;2025-09-02': datetime.time(7, 48, 5), '19792744;2025-09-01': datetime.time(7, 50, 29), '10239082;2025-09-05': datet

In [111]:
#%%Manipulación del Df_rellenar para que evalúe si la marca es mayor a la hora de entrada o menor a la hora de salida

# Convertir las columnas de tiempo a datetime para poder hacer comparaciones
# Nota: Las columnas programadas están en formato HH:MM, las marcas en HH:MM:SS
df_rellenar['Entrada_programada_dt'] = pd.to_datetime(df_rellenar['Entrada programada'], format='%H:%M', errors='coerce').dt.time
df_rellenar['Salida_programada_dt'] = pd.to_datetime(df_rellenar['Salida programada'], format='%H:%M', errors='coerce').dt.time
df_rellenar['PRIMERA_MARCA_dt'] = pd.to_datetime(df_rellenar['PRIMERA_MARCA'], format='%H:%M:%S', errors='coerce').dt.time
df_rellenar['ULTIMA_MARCA_dt'] = pd.to_datetime(df_rellenar['ULTIMA_MARCA'], format='%H:%M:%S', errors='coerce').dt.time

# Función para calcular diferencia en horas entre dos tiempos
def diferencia_horas(tiempo1, tiempo2):
    if pd.isna(tiempo1) or pd.isna(tiempo2):
        return np.nan
    
    # Convertir time a datetime para poder calcular diferencias
    fecha_base = pd.Timestamp('2000-01-01')
    dt1 = pd.Timestamp.combine(fecha_base.date(), tiempo1)
    dt2 = pd.Timestamp.combine(fecha_base.date(), tiempo2)
    
    # Calcular diferencia absoluta en horas
    diferencia = abs((dt2 - dt1).total_seconds() / 3600)
    return diferencia

# Calcular las diferencias
df_rellenar['Diferencia_entrada'] = df_rellenar.apply(
    lambda row: diferencia_horas(row['Entrada_programada_dt'], row['PRIMERA_MARCA_dt']), axis=1
)

df_rellenar['Diferencia_salida'] = df_rellenar.apply(
    lambda row: diferencia_horas(row['Salida_programada_dt'], row['ULTIMA_MARCA_dt']), axis=1
)

# Crear la columna "Observación de turno"
df_rellenar['Observación de turno'] = np.where(
    (df_rellenar['Diferencia_entrada'] >= 2) & (df_rellenar['Diferencia_salida'] >= 2),
    'Observación de turno',
    None
)

# Limpiar columnas auxiliares (opcional)
df_rellenar = df_rellenar.drop(columns=[
    'Entrada_programada_dt', 'Salida_programada_dt', 'PRIMERA_MARCA_dt', 'ULTIMA_MARCA_dt', 'Diferencia_entrada', 'Diferencia_salida'])

print("\nDataFrame después de agregar la columna 'Observación de turno':")
print(df_rellenar[['RUT', 'Fecha', 'Entrada programada', 'PRIMERA_MARCA', 'Salida programada', 'ULTIMA_MARCA', 'Observación de turno']].head(10))

print(f"\nRegistros con observación de turno: {df_rellenar['Observación de turno'].notna().sum()}")


DataFrame después de agregar la columna 'Observación de turno':
        RUT       Fecha Entrada programada PRIMERA_MARCA Salida programada  \
0  16418842  2025-09-04              07:50      07:57:00             17:00   
1  25322441  2025-09-04              07:50      08:14:51             17:00   
2  12857871  2025-09-04              07:50      07:07:13             17:00   
3  27064835  2025-09-04              07:50      07:03:43             17:00   
4  18906244  2025-09-04              07:50      07:52:38             17:00   
5  20053662  2025-09-04              07:50      08:10:13             17:00   
6  17841404  2025-09-04              07:50      08:04:49             17:00   
7  20761765  2025-09-04              07:50      08:12:51             17:00   
8  13546003  2025-09-04              07:50      13:56:44             17:00   
9   9093920  2025-09-04              07:50      08:01:42             17:00   

  ULTIMA_MARCA  Observación de turno  
0     15:59:11                  None 

In [112]:
display(df_rellenar)

,RUT,Nombre,Unidad,Puesto,Fecha,Tipo de turno,Entrada programada,Salida programada,Marca de entrada,Marca de salida,DV,Busqueda,KEY_IN,KEY_OUT,NO_KEY,PRIMERA_MARCA,ULTIMA_MARCA,Marca de entrada Generados,Marca de salida Generados,Observación de turno
0,16418842,Paulina González Muñoz,Customer,Administración Gral De Producción,2025-09-04,T,07:50,17:00,NaN,17:18:30,6,16418842;2025-09-04,0,0,6,07:57:00,15:59:11,07:57:00,17:18:30,None
1,25322441,Dariana Devies Castellano,Coordinador(a),Comercial Sabores,2025-09-04,T,07:50,17:00,NaN,17:02:37,K,25322441;2025-09-04,0,0,6,08:14:51,16:25:35,08:14:51,17:02:37,None
2,12857871,Sixto Beiza Mardones,Operario,Esencias y Env. Sabores,2025-09-04,T,07:50,17:00,NaN,16:50:29,4,12857871;2025-09-04,0,1,9,07:07:13,16:50:49,07:07:13,16:50:29,None
3,27064835,Germán Leyes Portilla,Operario,Planta Fragancias,2025-09-04,T,07:50,17:00,NaN,16:46:06,5,27064835;2025-09-04,0,1,5,07:03:43,16:46:06,07:03:43,16:46:06,None
4,18906244,Marco Rojas Mora,Operario,Planta Fragancias,2025-09-04,T,07:50,17:00,NaN,16:46:30,3,18906244;2025-09-04,0,1,7,07:52:38,16:46:58,07:52:38,16:46:30,None
5,20053662,Jazmín Castillo Ponce,Asistente,Desarrollo Sabores Salados,2025-09-04,T,07:50,17:00,NaN,17:01:18,2,20053662;2025-09-04,0,1,11,08:10:13,17:02:03,08:10:13,17:01:18,None
6,17841404,Fernando Magaña Cancino,Especialista,Desarrollo Sabores Salados,2025-09-04,T,07:50,17:00,NaN,17:00:58,6,17841404;2025-09-04,0,0,4,08:04:49,14:30:58,08:04:49,17:00:58,None
7,20761765,Alejandro Ayala Salas,Operario,Ingeniería y Medio Ambiente,2025-09-04,T,07:50,17:00,NaN,17:00:09,2,20761765;2025-09-04,0,1,7,08:12:51,17:01:19,08:12:51,17:00:09,None
8,13546003,Jose San Martin Silva,Jefatura,Aseguramiento De Calidad,2025-09-04,T,07:50,17:00,NaN,18:11:21,6,13546003;2025-09-04,0,0,5,13:56:44,14:55:10,13:56:44,18:11:21,Observación de turno
9,9093920,Ximena Guerrero Quezada,Jefatura,Desarrollo Fragancias,2025-09-04,T,07:50,17:00,NaN,17:10:57,3,9093920;2025-09-04,0,0,5,08:01:42,13:30:28,08:01:42,17:10:57,None


In [113]:
# Analizar el formato de las columnas de entrada y salida programada
print("Muestra de las columnas de tiempo:")
print(df_rellenar[['Entrada programada', 'Salida programada', 'PRIMERA_MARCA', 'ULTIMA_MARCA']].head(10))
print("\nTipos de datos:")
print(df_rellenar[['Entrada programada', 'Salida programada', 'PRIMERA_MARCA', 'ULTIMA_MARCA']].dtypes)
print("\nEjemplos únicos de 'Entrada programada':")
print(df_rellenar['Entrada programada'].unique()[:10])
print("\nEjemplos únicos de 'Salida programada':")
print(df_rellenar['Salida programada'].unique()[:10])

Muestra de las columnas de tiempo:
  Entrada programada Salida programada PRIMERA_MARCA ULTIMA_MARCA
0              07:50             17:00      07:57:00     15:59:11
1              07:50             17:00      08:14:51     16:25:35
2              07:50             17:00      07:07:13     16:50:49
3              07:50             17:00      07:03:43     16:46:06
4              07:50             17:00      07:52:38     16:46:58
5              07:50             17:00      08:10:13     17:02:03
6              07:50             17:00      08:04:49     14:30:58
7              07:50             17:00      08:12:51     17:01:19
8              07:50             17:00      13:56:44     14:55:10
9              07:50             17:00      08:01:42     13:30:28

Tipos de datos:
Entrada programada    object
Salida programada     object
PRIMERA_MARCA         object
ULTIMA_MARCA          object
dtype: object

Ejemplos únicos de 'Entrada programada':
['07:50' '14:00' '05:50' '06:30' '06:50']

Ejemplo

In [114]:
#%% Exportar archivo en Excel con UTF-8
import datetime
now = datetime.datetime.now()
timestamp_str = now.strftime("%Y-%m-%d_%H-%M-%S") 
titulo = "df_rellenar_con_observaciones"
nombre_archivo = f'C:/Users/bgacitua/Desktop/Reportes Corrección de Marcas/Planillas_Python_Spyder/{titulo}_{timestamp_str}.xlsx'

# Exportar a Excel con formato UTF-8
df_rellenar.to_excel(nombre_archivo, sheet_name='Reporte_Marcas', index=False, engine='openpyxl')

print(f"\n¡Reporte exportado exitosamente en formato Excel a '{nombre_archivo}'!")
print(f"Registros exportados: {len(df_rellenar)}")
print(f"Columnas incluidas: {list(df_rellenar.columns)}")
print(f"Registros con observación de turno: {df_rellenar['Observación de turno'].notna().sum()}")


¡Reporte exportado exitosamente en formato Excel a 'C:/Users/bgacitua/Desktop/Reportes Corrección de Marcas/Planillas_Python_Spyder/df_rellenar_con_observaciones_2025-09-05_08-03-37.xlsx'!
Registros exportados: 46
Columnas incluidas: ['RUT', 'Nombre', 'Unidad', 'Puesto', 'Fecha', 'Tipo de turno', 'Entrada programada', 'Salida programada', 'Marca de entrada', 'Marca de salida', 'DV', 'Busqueda', 'KEY_IN', 'KEY_OUT', 'NO_KEY', 'PRIMERA_MARCA', 'ULTIMA_MARCA', 'Marca de entrada Generados', 'Marca de salida Generados', 'Observación de turno']
Registros con observación de turno: 5


In [115]:
# %%
# --- 4. Cerrar la Conexión (al finalizar todas las operaciones) ---
if 'cnxn' in locals() and cnxn: # Verificar si 'cnxn' existe y no es None
    try:
        cnxn.close()
        print("\nConexión a SQL Server cerrada exitosamente.")
    except Exception as e:
        print(f"Error al cerrar la conexión: {e}")
else:
    print("La conexión no fue establecida o ya está cerrada.")


Conexión a SQL Server cerrada exitosamente.


In [116]:
#Importar dataframe de marcas no asignadas
reporte_marcas_no_asignadas = pd.read_excel(path_marcas_sin_turno)

In [117]:
print("DataFrame original:")
print(reporte_marcas_no_asignadas.head())

# Transformar valores "-" en NaN en la columna "RUT Trabajador"
print(f"\nAntes de la transformación:")
print(f"Valores únicos en 'RUT Trabajador': {reporte_marcas_no_asignadas['RUT Trabajador'].unique()[:10]}")
print(f"Cantidad de valores '-': {(reporte_marcas_no_asignadas['RUT Trabajador'] == '-').sum()}")

# Reemplazar "-" con NaN
reporte_marcas_no_asignadas['RUT Trabajador'] = reporte_marcas_no_asignadas['RUT Trabajador'].replace('-', np.nan)

print(f"\nDespués de la transformación:")
print(f"Valores únicos en 'RUT Trabajador': {reporte_marcas_no_asignadas['RUT Trabajador'].unique()[:10]}")
print(f"Cantidad de valores NaN: {reporte_marcas_no_asignadas['RUT Trabajador'].isna().sum()}")

print("\nDataFrame después de la transformación:")
display(reporte_marcas_no_asignadas.head())

# Eliminar filas con valores NaN en la columna "RUT Trabajador"
print(f"\nAntes de eliminar filas con NaN:")
print(f"Total de registros: {len(reporte_marcas_no_asignadas)}")
print(f"Registros con NaN en 'RUT Trabajador': {reporte_marcas_no_asignadas['RUT Trabajador'].isna().sum()}")

# Eliminar filas donde 'RUT Trabajador' es NaN
reporte_marcas_no_asignadas = reporte_marcas_no_asignadas.dropna(subset=['RUT Trabajador'])

print(f"\nDespués de eliminar filas con NaN:")
print(f"Total de registros: {len(reporte_marcas_no_asignadas)}")
print(f"Registros con NaN en 'RUT Trabajador': {reporte_marcas_no_asignadas['RUT Trabajador'].isna().sum()}")

print("\nDataFrame final limpio:")
display(reporte_marcas_no_asignadas.head())



DataFrame original:
  RUT Trabajador               Nombre  \
0            NaN                        
1     14509971-4         Ruben Diaz .   
2     19287701-6  Valeria Cajales Ruz   
3     17253721-9   Erik Moreno Corral   
4            NaN                        

                                    Unidad            Puesto Fecha Marca  \
0                                      NaN               NaN  04/09/2025   
1  Bodega Recepción, Despacho y Transporte  Chofer y Peoneta  04/09/2025   
2                 Sabores Bebidas y Dulces      Especialista  04/09/2025   
3  Bodega Recepción, Despacho y Transporte  Chofer y Peoneta  04/09/2025   
4                                      NaN               NaN  04/09/2025   

  Hora Marca Sentido Marca  
0   05:42:06             E  
1   07:18:22             E  
2   07:48:09             E  
3   07:45:05             E  
4   07:46:15             E  

Antes de la transformación:
Valores únicos en 'RUT Trabajador': [nan '14509971-4' '19287701-6' '17253

,RUT Trabajador,Nombre,Unidad,Puesto,Fecha Marca,Hora Marca,Sentido Marca
0,NaN,,NaN,NaN,04/09/2025,05:42:06,E
1,14509971-4,Ruben Diaz .,"Bodega Recepción, Despacho y Transporte",Chofer y Peoneta,04/09/2025,07:18:22,E
2,19287701-6,Valeria Cajales Ruz,Sabores Bebidas y Dulces,Especialista,04/09/2025,07:48:09,E
3,17253721-9,Erik Moreno Corral,"Bodega Recepción, Despacho y Transporte",Chofer y Peoneta,04/09/2025,07:45:05,E
4,NaN,,NaN,NaN,04/09/2025,07:46:15,E



Antes de eliminar filas con NaN:
Total de registros: 32
Registros con NaN en 'RUT Trabajador': 16

Después de eliminar filas con NaN:
Total de registros: 16
Registros con NaN en 'RUT Trabajador': 0

DataFrame final limpio:


,RUT Trabajador,Nombre,Unidad,Puesto,Fecha Marca,Hora Marca,Sentido Marca
1,14509971-4,Ruben Diaz .,"Bodega Recepción, Despacho y Transporte",Chofer y Peoneta,04/09/2025,07:18:22,E
2,19287701-6,Valeria Cajales Ruz,Sabores Bebidas y Dulces,Especialista,04/09/2025,07:48:09,E
3,17253721-9,Erik Moreno Corral,"Bodega Recepción, Despacho y Transporte",Chofer y Peoneta,04/09/2025,07:45:05,E
5,21129006-4,Bayron Torres Otarola,Esencias y Env. Sabores,Operario,04/09/2025,07:42:08,S
6,15776698-8,María Contreras Diaz,Recursos Humanos,Diseñadora,04/09/2025,08:00:52,E


In [118]:
reporte_marcas_no_asignadas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 1 to 31
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   RUT Trabajador  16 non-null     object
 1   Nombre          16 non-null     object
 2   Unidad          16 non-null     object
 3   Puesto          16 non-null     object
 4   Fecha Marca     16 non-null     object
 5   Hora Marca      16 non-null     object
 6   Sentido Marca   16 non-null     object
dtypes: object(7)
memory usage: 1.0+ KB


In [119]:
# Separar la columna "RUT Trabajador" por el delimitador "-"
print(f"\n🔧 SEPARANDO RUT Y DÍGITO VERIFICADOR:")
print(f"Procesando columna 'RUT Trabajador'...")

# Verificar si contiene el delimitador "-"
tiene_delimitador = reporte_marcas_no_asignadas['RUT Trabajador'].astype(str).str.contains('-', na=False)

print(f"Registros con delimitador '-': {tiene_delimitador.sum()}")
print(f"Total de registros: {len(reporte_marcas_no_asignadas)}")

if tiene_delimitador.any():
    # Separar RUT y DV usando el delimitador "-"
    rut_split = reporte_marcas_no_asignadas['RUT Trabajador'].astype(str).str.split('-', expand=True)
    
    # Actualizar la columna 'RUT Trabajador' con solo la parte numérica
    reporte_marcas_no_asignadas['RUT Trabajador'] = rut_split[0]
    
    # Crear la columna 'DV' con el dígito verificador
    reporte_marcas_no_asignadas['DV'] = rut_split[1].fillna('')
    
    print(f"✅ Separación completada:")
    print(f"   • Registros procesados: {tiene_delimitador.sum()}")
    print(f"   • RUT Trabajador: solo números")
    print(f"   • DV: nueva columna creada")
    
    # Mostrar ejemplos de la separación
    print(f"\n📋 Ejemplos de separación:")
    ejemplos_separacion = reporte_marcas_no_asignadas[['RUT Trabajador', 'DV']].head()
    display(ejemplos_separacion)
    
else:
    # Si no hay delimitador, crear columna DV vacía
    reporte_marcas_no_asignadas['DV'] = ''
    print("ℹ️  No se encontraron delimitadores '-' en la columna 'RUT Trabajador'")

# Convertir RUT Trabajador a numérico para mejor manejo
try:
    reporte_marcas_no_asignadas['RUT Trabajador'] = pd.to_numeric(
        reporte_marcas_no_asignadas['RUT Trabajador'], errors='coerce'
    )
    print(f"✅ Columna 'RUT Trabajador' convertida a tipo numérico")
except Exception as e:
    print(f"⚠️  Advertencia al convertir 'RUT Trabajador' a numérico: {e}")

print(f"\n📊 RESULTADO FINAL:")
print(f"DataFrame con RUT y DV separados:")
display(reporte_marcas_no_asignadas.head())

# Convertir formato de fecha de dd/mm/yyyy a yyyy-mm-dd
print(f"\n📅 CONVIRTIENDO FORMATO DE FECHA:")
print(f"Procesando columna 'Fecha Marca' desde dd/mm/yyyy hacia yyyy-mm-dd...")

# Mostrar formato actual
print(f"\nAntes de la conversión:")
print(f"Tipo de dato: {reporte_marcas_no_asignadas['Fecha Marca'].dtype}")
print(f"Ejemplos de fechas actuales:")
print(reporte_marcas_no_asignadas['Fecha Marca'].head())

try:
    # Convertir de dd/mm/yyyy a datetime
    reporte_marcas_no_asignadas['Fecha Marca'] = pd.to_datetime(
        reporte_marcas_no_asignadas['Fecha Marca'], 
        format='%d/%m/%Y', 
        errors='coerce'
    )
    
    # Convertir a formato yyyy-mm-dd (string)
    reporte_marcas_no_asignadas['Fecha Marca'] = reporte_marcas_no_asignadas['Fecha Marca'].dt.strftime('%Y-%m-%d')
    
    print(f"\n✅ Conversión completada exitosamente")
    print(f"Nuevo tipo de dato: {reporte_marcas_no_asignadas['Fecha Marca'].dtype}")
    print(f"Ejemplos de fechas convertidas:")
    print(reporte_marcas_no_asignadas['Fecha Marca'].head())
    
    # Verificar si hay fechas que no pudieron convertirse
    fechas_nulas = reporte_marcas_no_asignadas['Fecha Marca'].isna().sum()
    if fechas_nulas > 0:
        print(f"⚠️  {fechas_nulas} fechas no pudieron convertirse y quedaron como NaN")
    else:
        print(f"🎉 Todas las fechas se convirtieron correctamente")
        
except Exception as e:
    print(f"❌ Error al convertir fechas: {str(e)}")
    print(f"💡 Verifica que las fechas estén en formato dd/mm/yyyy")

# Mostrar muestra final del DataFrame
print(f"\n🔍 MUESTRA FINAL DEL DATAFRAME:")
columnas_clave = ['RUT Trabajador', 'DV', 'Fecha Marca', 'Hora Marca']
columnas_disponibles = [col for col in columnas_clave if col in reporte_marcas_no_asignadas.columns]
display(reporte_marcas_no_asignadas[columnas_disponibles].head())

print(f"\n📋 RESUMEN DEL PROCESAMIENTO:")
print(f"   • Total de registros: {len(reporte_marcas_no_asignadas)}")
print(f"   • RUT Trabajador: formato numérico")
print(f"   • DV: dígito verificador separado")
print(f"   • Fecha Marca: formato yyyy-mm-dd")
print(f"   • Datos listos para análisis ✅")


🔧 SEPARANDO RUT Y DÍGITO VERIFICADOR:
Procesando columna 'RUT Trabajador'...
Registros con delimitador '-': 16
Total de registros: 16
✅ Separación completada:
   • Registros procesados: 16
   • RUT Trabajador: solo números
   • DV: nueva columna creada

📋 Ejemplos de separación:


,RUT Trabajador,DV
1,14509971,4
2,19287701,6
3,17253721,9
5,21129006,4
6,15776698,8


✅ Columna 'RUT Trabajador' convertida a tipo numérico

📊 RESULTADO FINAL:
DataFrame con RUT y DV separados:


,RUT Trabajador,Nombre,Unidad,Puesto,Fecha Marca,Hora Marca,Sentido Marca,DV
1,14509971,Ruben Diaz .,"Bodega Recepción, Despacho y Transporte",Chofer y Peoneta,04/09/2025,07:18:22,E,4
2,19287701,Valeria Cajales Ruz,Sabores Bebidas y Dulces,Especialista,04/09/2025,07:48:09,E,6
3,17253721,Erik Moreno Corral,"Bodega Recepción, Despacho y Transporte",Chofer y Peoneta,04/09/2025,07:45:05,E,9
5,21129006,Bayron Torres Otarola,Esencias y Env. Sabores,Operario,04/09/2025,07:42:08,S,4
6,15776698,María Contreras Diaz,Recursos Humanos,Diseñadora,04/09/2025,08:00:52,E,8



📅 CONVIRTIENDO FORMATO DE FECHA:
Procesando columna 'Fecha Marca' desde dd/mm/yyyy hacia yyyy-mm-dd...

Antes de la conversión:
Tipo de dato: object
Ejemplos de fechas actuales:
1    04/09/2025
2    04/09/2025
3    04/09/2025
5    04/09/2025
6    04/09/2025
Name: Fecha Marca, dtype: object

✅ Conversión completada exitosamente
Nuevo tipo de dato: object
Ejemplos de fechas convertidas:
1    2025-09-04
2    2025-09-04
3    2025-09-04
5    2025-09-04
6    2025-09-04
Name: Fecha Marca, dtype: object
🎉 Todas las fechas se convirtieron correctamente

🔍 MUESTRA FINAL DEL DATAFRAME:


,RUT Trabajador,DV,Fecha Marca,Hora Marca
1,14509971,4,2025-09-04,07:18:22
2,19287701,6,2025-09-04,07:48:09
3,17253721,9,2025-09-04,07:45:05
5,21129006,4,2025-09-04,07:42:08
6,15776698,8,2025-09-04,08:00:52



📋 RESUMEN DEL PROCESAMIENTO:
   • Total de registros: 16
   • RUT Trabajador: formato numérico
   • DV: dígito verificador separado
   • Fecha Marca: formato yyyy-mm-dd
   • Datos listos para análisis ✅
